<a href="https://colab.research.google.com/github/cristinarainich/Sternberg_analysis/blob/main/Environmental.%20Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

Updated 17/05 12:59

In [165]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io
from pathlib import Path

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental')

In [163]:
!ls /content/gdrive/MyDrive/CoganLab/Environmental

 D29_Block_1_TrialData.mat	  D85_Block_1_TrialData.mat
 D64_Block_1_TrialData.mat	  D86_Block_1_TrialData.mat
 D65_Block_1_TrialData_1.mat	  D87_Block_1_TrialData.mat
 D65_Block_1_TrialData.mat	  eeg_envir.csv
'D71_Block_1_TrialData (1).mat'   eeg_envir_processed.csv
'D73_Block_1_TrialData (2).mat'   Environmental_Subject.mat
'D77_Block_1_TrialData (1).mat'   healthy
 D80_Block_1_TrialData.mat	  ieeg_envir.csv
 D81_Block_1_TrialData.mat	  ieeg_envir_processed.csv
 D84_Block_1_TrialData.mat	  stimuli


In [166]:
# Patients
ieeg = scipy.io.loadmat(
    working_path.joinpath('Environmental_Subject.mat')
    )

subj1 = scipy.io.loadmat(
    working_path.joinpath('D65_Block_1_TrialData.mat')
    )

subj65 = scipy.io.loadmat(
    working_path.joinpath('D65_Block_1_TrialData_1.mat')
    )

subj64 = scipy.io.loadmat(
    working_path.joinpath('D64_Block_1_TrialData.mat')
)
subj73 = scipy.io.loadmat(
    working_path.joinpath('D73_Block_1_TrialData (2).mat')
    )
subj77 = scipy.io.loadmat(
    working_path.joinpath('D77_Block_1_TrialData (1).mat')
    )
subj80 = scipy.io.loadmat(
    working_path.joinpath('D80_Block_1_TrialData.mat')
    )
subj81 = scipy.io.loadmat(
    working_path.joinpath('D81_Block_1_TrialData.mat')
    )
subj84 = scipy.io.loadmat(
    working_path.joinpath('D84_Block_1_TrialData.mat')
    )
subj85 = scipy.io.loadmat(
    working_path.joinpath('D85_Block_1_TrialData.mat')
    )
subj86 = scipy.io.loadmat(
    working_path.joinpath('D86_Block_1_TrialData.mat')
    )
subj87 = scipy.io.loadmat(
    working_path.joinpath('D87_Block_1_TrialData.mat')
    )


OSError: ignored

In [6]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental/healthy')

In [7]:
# healthy
files = [] 
begin = 'E'
end = '_Block_1_TrialData.mat'
files.append(begin + '32' + end)
for i in range(34, 47):
  files.append(begin + str(i) + end)

In [8]:
files[0]

'E32_Block_1_TrialData.mat'

In [9]:
eeg = []
for i in files:
  subset = scipy.io.loadmat(
      working_path.joinpath(i)
  )
  eeg.append(subset)

#Working with stimuli

In [47]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental/stimuli')

In [48]:
!ls /content/gdrive/MyDrive/CoganLab/Environmental/stimuli

ieeg_envir.csv		 proposed_nw3.mat  proposed_w3_global.mat  untrn_nw.mat
imaginability.txt	 proposed_nw.mat   proposed_w3.mat	   untrn_w.mat
proposed_nw3_global.mat  proposed_w2.mat   proposed_w.mat


In [49]:
stimuli_words = scipy.io.loadmat(
    working_path.joinpath('proposed_w3.mat')
)

In [50]:
columns = stimuli_words['proposed_w'].dtype.names
stim_words = pd.DataFrame(columns=columns)
for i in range(0, stimuli_words['proposed_w'].shape[1]):
  values = []
  subset = stimuli_words['proposed_w'][0][i]
  for name in columns:
    values.append(subset[name])
  stim_words.loc[len(stim_words.index)] = values

In [51]:
stimuli = []
for i in stim_words.Word.values:
  stimuli.append(i[0])

freq = []
for i in stim_words.SFreq.values:
  freq.append(i[0][0])

imagine = []
for i in stim_words.imaginability.values:
  imagine.append(i[0][0])

category = []
for i in range(0, 24):
  category.append('word')

In [52]:
columns = ['Category', 'Stimulus', 'Frequency', 'Imaginability']
words = pd.DataFrame(columns=columns)
words['Category'] = category
words['Stimulus'] = stimuli
words['Frequency'] = freq
words['Imaginability'] = imagine

In [53]:
stimuli_nonwords = scipy.io.loadmat(
    working_path.joinpath('proposed_nw3.mat')
)

In [54]:
columns = stimuli_nonwords['proposed_nw'].dtype.names
stim_nonwords = pd.DataFrame(columns=columns)
for i in range(0, stimuli_nonwords['proposed_nw'].shape[1]):
  values = []
  subset = stimuli_nonwords['proposed_nw'][0][i]
  for name in columns:
    values.append(subset[name])
  stim_nonwords.loc[len(stim_nonwords.index)] = values

In [55]:
stimuli = []
for i in stim_nonwords.Word.values:
  stimuli.append(i[0])

freq = []
for i in stim_nonwords.SFreq.values:
  freq.append(i[0][0])

category = []
for i in range(0, 24):
  category.append('nonword')

In [56]:
columns = ['Category', 'Stimulus', 'Frequency', 'Imaginability']
nonwords = pd.DataFrame(columns=columns)
nonwords['Category'] = category
nonwords['Stimulus'] = stimuli
nonwords['Frequency'] = freq
nonwords['Imaginability'] = np.nan

In [57]:
all_stim = pd.concat([words, nonwords])

# iEEG dataset

In [10]:
data_ieeg = ieeg['Subject']
print('Number of subjects in iEEG dataset is', data_ieeg.shape[1])

Number of subjects in iEEG dataset is 19


In [11]:
subjects_ieeg = [] # subject numbers
for i in data_ieeg['Name'][0]:
  subjects_ieeg.append(i[0])
print(subjects_ieeg)

['D27', 'D28', 'D30', 'D31', 'D32', 'D35', 'D37', 'D38', 'D39', 'D41', 'D42', 'D47', 'D53', 'D54', 'D55', 'D57', 'D63', 'D69', 'D71']


In [12]:
columns = []
for i in range(0, data_ieeg.shape[1]):
  actual = data_ieeg[0][i]['trialInfo'][0][0].dtype.names
  for name in actual:
    if name not in columns:
      columns.append(name)

In [13]:
def get_to_df(data):
  df = pd.DataFrame(columns=columns)
  df['Trial'] = np.nan
  df['Subject'] = np.nan

  for i in range(0, data.shape[1]):
    # for each subject
    subset = data[0][i]
    subject = subset['Name'][0]
    print(subset['Task'])
    print(subject)
    for trial in range(0, subset['trialInfo'].shape[1]):
      # for each trial
      sub = subset['trialInfo'][0][trial]
      dtype = sub.dtype.names
      values = []
      for name in columns:
        if name in dtype:
          values.append(sub[name])
        else:
          values.append(np.nan)
      values.append(str(trial)) # adding a trial number
      values.append(subject)
      df.loc[len(df.index)] = values
  return df

In [14]:
ieeg = get_to_df(data_ieeg)

['Environmental_Sternberg']
D27
['Environmental_Sternberg']
D28
['Environmental_Sternberg']
D30
['Environmental_Sternberg']
D31
['Environmental_Sternberg']
D32
['Environmental_Sternberg']
D35
['Environmental_Sternberg']
D37
['Environmental_Sternberg']
D38
['Environmental_Sternberg']
D39
['Environmental_Sternberg']
D41
['Environmental_Sternberg']
D42
['Environmental_Sternberg']
D47
['Environmental_Sternberg']
D53
['Environmental_Sternberg']
D54
['Environmental_Sternberg']
D55
['Environmental_Sternberg']
D57
['Environmental_Sternberg']
D63
['Environmental_Sternberg']
D69
[]
D71


In [15]:
# adding subjects D64
df_64 = pd.DataFrame(columns=columns)
df_64['Trial'] = np.nan
df_64['Subject'] = np.nan
for trial in range(0, subj64['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj64['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D64')
  df_64.loc[len(df_64.index)] = values

In [16]:
# adding subjects D65
df_65 = pd.DataFrame(columns=columns)
df_65['Trial'] = np.nan
df_65['Subject'] = np.nan
for trial in range(0, subj1['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj1['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D65')
  df_65.loc[len(df_65.index)] = values
  i = 96
for trial in range(0, subj65['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj65['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(i)) # adding a trial number
  values.append('D65')
  i += 1
  df_65.loc[len(df_65.index)] = values

In [17]:
ieeg = pd.concat([ieeg, df_64])
ieeg = pd.concat([ieeg, df_65])

In [18]:
# adding subject 73
df_73 = pd.DataFrame(columns=columns)
df_73['Trial'] = np.nan
df_73['Subject'] = np.nan
for trial in range(0, subj73['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj73['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D73')
  df_73.loc[len(df_73.index)] = values

In [19]:
# adding subject 77
df_77 = pd.DataFrame(columns=columns)
df_77['Trial'] = np.nan
df_77['Subject'] = np.nan
for trial in range(0, subj77['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj77['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D77')
  df_77.loc[len(df_77.index)] = values

In [20]:
ieeg = pd.concat([ieeg, df_73])
ieeg = pd.concat([ieeg, df_77])

In [21]:
ieeg.groupby(['Subject'])['Trial'].count()

Subject
D27    160
D28    120
D30    168
D31    168
D32    168
D35    168
D37    168
D38    160
D39    168
D41    168
D42    168
D47    168
D53    168
D54    168
D55    168
D57    168
D63    168
D64    168
D65    168
D69    168
D71    168
D73    168
D77    168
Name: Trial, dtype: int64

In [22]:
# saving ieeg dataset
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental')
ieeg.to_csv(
    Path(working_path).joinpath('ieeg_envir.csv')
)

# Preprocessing of iEEG dataset

In [ ]:
ieeg = pd.read_csv(
    working_path.joinpath('ieeg_envir.csv')
    )

In [23]:
ieeg_alter = pd.read_csv(
    working_path.joinpath('ieeg_envir_processed.csv')
    )

In [24]:
ieeg_alter = ieeg_alter.drop('Unnamed: 0', axis=1)

In [25]:
ieeg.Subject.unique()

array(['D27', 'D28', 'D30', 'D31', 'D32', 'D35', 'D37', 'D38', 'D39',
       'D41', 'D42', 'D47', 'D53', 'D54', 'D55', 'D57', 'D63', 'D69',
       'D71', 'D64', 'D65', 'D73', 'D77'], dtype=object)

In [87]:
ieeg_ = ieeg[['block', 'ProbeType', 'StimlusCategory',
       'ReactionTime', 'Resp', 'RespCorrect', 'Omission',
       'stimulusSounds_idx', 'stimulusSounds_name', 'ProbeCategory', 'probeSound_idx',
       'probeSound_name', 'StimulusCategory', 'Trial',
       'Subject']]

In [88]:
# looks like D27 performed Neighbourhood here
# let's exclude them
ieeg_ = ieeg_[ieeg_['Subject'] != 'D27']

In [89]:
ieeg_ = ieeg_.drop(['Omission', 'ProbeType'], axis=1)

In [91]:
# working on the columns to make them more convinient to use
block = ieeg_.block.values
new_block = []
for i in block:
  while type(i) is np.ndarray:
    i = i[0]
  new_block.append(i)

In [92]:
rt = ieeg_.ReactionTime.values
new_rt = []
for i in rt:
  while type(i) is np.ndarray:
    i = i[0]
  new_rt.append(i)

In [93]:
acc = ieeg_.RespCorrect.values
new_acc = []
for i in acc:
  while type(i) is np.ndarray:
    i = i[0]
  new_acc.append(i)

In [94]:
resp = ieeg_.Resp.values
new_resp = []
for i in resp:
  while type(i) is np.ndarray:
    i = i[0]
  new_resp.append(i)

In [95]:
probe = ieeg_.ProbeCategory.values
new_probe = []
for i in probe:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe.append(i)

In [96]:
probe_idx = ieeg_.probeSound_idx.values
new_probe_idx = []
for i in probe_idx:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_idx.append(i)

In [97]:
stim_idx = ieeg_.stimulusSounds_idx.values
new_stim_idx = []
for i in stim_idx:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0])
  new_stim_idx.append(element)

In [98]:
probe_name = ieeg_.probeSound_name.values
new_probe_name = []
for i in probe_name:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_name.append(i)

In [131]:
stim_name = ieeg_.stimulusSounds_name.values
new_stim_name = []
for i in stim_name:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    new_j = j[0][0]
    if type(new_j) is np.str_:
      element.append(new_j)
    else: 
      element.append(new_j[0])
  new_stim_name.append(element)

In [134]:
stim1 = ieeg_.StimlusCategory.values
stim2 = ieeg_.StimulusCategory.values
new_stim = []
for i in range(0, ieeg_.shape[0]):
  if type(stim1[i]) is np.ndarray:
    element = stim1[i][0][0][0]
    new_stim.append(element)
  elif type(stim2[i]) is np.ndarray:
    element = stim2[i][0][0][0]
    new_stim.append(element)
  else:
    print(stim1[i], stim2[i])
    break

In [135]:
ieeg_['block'] = new_block
ieeg_['ReactionTime'] = new_rt
ieeg_['RespCorrect'] = new_acc
ieeg_['Resp'] = new_resp
ieeg_['ProbeCategory'] = new_probe
ieeg_['probeSound_idx'] = new_probe_idx
ieeg_['stimulusSounds_idx'] = new_stim_idx
ieeg_['probeSound_name'] = new_probe_name
ieeg_['stimulusSounds_name'] = new_stim_name
ieeg_['StimulusCategory'] = new_stim
ieeg_ = ieeg_.drop('StimlusCategory', axis=1)

In [136]:
length = []
for i in ieeg_.stimulusSounds_idx.values:
  if len(i) not in length:
    length.append(len(i))
print(length)

[3, 7, 9, 5]


In [137]:
length = []
for i in ieeg_.stimulusSounds_idx.values:
  length.append(len(i))
ieeg_['Length'] = length

In [138]:
words = []
nonwords = []
environment = []
for i in ieeg_.StimulusCategory.values:
  if i == 'nonwords':
    words.append(0)
    nonwords.append(1)
    environment.append(0)
  elif i == 'words':
    words.append(1)
    nonwords.append(0)
    environment.append(0)
  elif i == 'environment':
    words.append(0)
    nonwords.append(0)
    environment.append(1)

In [139]:
ieeg_['Words'] = words
ieeg_['Nonwords'] = nonwords
ieeg_['Environment'] = environment

In [140]:
ieeg_.groupby(['Subject'])['RespCorrect'].mean()

Subject
D28    0.750000
D30    0.785714
D31    0.815476
D32    0.779762
D35    0.833333
D37    0.672619
D38    0.800000
D39    0.755952
D41    0.428571
D42    0.875000
D47    0.791667
D53    0.851190
D54    0.488095
D55    0.839286
D57    0.880952
D63    0.738095
D64    0.803571
D65    0.642857
D69    0.732143
D71    0.821429
D73    0.773810
D77    0.779762
Name: RespCorrect, dtype: float64

In [141]:
# it looks like D28 has the very weird categories and sounds recorded
ieeg_ = ieeg_[ieeg_['Subject'] != 'D28']
ieeg_ = ieeg_[ieeg_['Subject'] != 'D30']
ieeg_ = ieeg_[ieeg_['Subject'] != 'D31']
ieeg_ = ieeg_[ieeg_['Subject'] != 'D37']
ieeg_ = ieeg_[ieeg_['Subject'] != 'D41']
ieeg_ = ieeg_[ieeg_['Subject'] != 'D54']
ieeg_ = ieeg_[ieeg_['Subject'] != 'D65']

In [155]:
new_stim = []
for i in all_stim.Stimulus.values:
  new_stim.append(i + '.wav')
all_stim['Stimulus'] = new_stim

In [158]:
category = ieeg_.StimulusCategory.values.tolist()
probe = ieeg_.probeSound_name.values
stimuli = ieeg_.stimulusSounds_name.values
standard = all_stim[all_stim['Category'] == 'word']['Stimulus'].values.tolist()
standard_freq = all_stim[all_stim['Category'] == 'word']['Frequency'].values.tolist()
standard_imag = all_stim[all_stim['Category'] == 'word']['Imaginability'].values.tolist()


stim_freq = []
stim_imag = []
probe_freq = []
probe_imag = []

for i in range(0, ieeg_.shape[0]):
  if category[i] == 'words':
    for name in standard:
      if probe[i] == name:
        freq = standard_freq[standard.index(name)]
        imag = standard_imag[standard.index(name)]
        probe_freq.append(freq)
        probe_imag.append(imag)

    values_freq = []
    values_imag = []      
    for lis in stimuli[i]:
      for name in standard:
        if lis == name:
          freq = standard_freq[standard.index(name)]
          imag = standard_imag[standard.index(name)]
          values_freq.append(freq)
          values_imag.append(imag)
    stim_freq.append(values_freq)
    stim_imag.append(values_imag)
  
  else:
    stim_freq.append(np.nan)
    stim_imag.append(np.nan)
    probe_freq.append(np.nan)
    probe_imag.append(np.nan)

In [159]:
print(ieeg_.shape)
print(len(stim_freq))
print(len(stim_imag))
print(len(probe_freq))
print(len(probe_imag))

(2512, 16)
2512
2512
2512
2512


In [160]:
ieeg_['ProbeFreq'] = probe_freq
ieeg_['ProbeImag'] = probe_imag
ieeg_['StimFreq'] = stim_freq
ieeg_['StimImag'] = stim_imag

In [ ]:
ieeg_.to_csv(
    Path(working_path).joinpath('ieeg_envir_processed.csv')
)

# EEG dataset

In [ ]:
subjects = []
for i in files:
  subjects.append(i[:3])

In [ ]:
columns = []
for subject in eeg:
  # for each subject
  actual = subject['trialInfo'][0][0].dtype.names
  for name in actual:
    if name not in columns:
      columns.append(name)

In [ ]:
df_ = pd.DataFrame(columns=columns)
df_['Trial'] = np.nan
df_['Subject'] = np.nan
for subject in eeg:
  for trial in range(0, subject['trialInfo'].shape[1]):
    # for each trial
    values = []
    subset = subject['trialInfo'][0][trial]
    dtype = subset.dtype.names
    for name in columns:
      if name in dtype:
        values.append(subset[name])
      else:
        print('check')
        values.append(np.nan)
    values.append(str(trial))
    values.append(subjects[eeg.index(subject)])
    df_.loc[len(df_.index)] = values

In [ ]:
eeg = df_.copy()

In [ ]:
eeg.groupby(['Subject'])['Trial'].count()

Subject
E32    216
E34    216
E35    216
E36    216
E37    216
E38    216
E39    216
E40    184
E41    216
E42    216
E43    216
E44    216
E45    216
E46    216
Name: Trial, dtype: int64

In [ ]:
# saving eeg file 
eeg.to_csv(
    Path(working_path).joinpath('eeg_envir.csv')
)

#Preprocessing of EEG dataset

In [ ]:
eeg = pd.read_csv(
    working_path.joinpath('eeg_envir.csv')
    )

In [ ]:
eeg.Subject.unique()

array(['E32', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40', 'E41',
       'E42', 'E43', 'E44', 'E45', 'E46'], dtype=object)

In [ ]:
len(eeg.Subject.unique())

14

In [ ]:
eeg_ = eeg[[
    'block', 'StimulusCategory', 'ReactionTime',
       'Resp', 'RespCorrect', 'stimulusSounds_idx',
       'ProbeCategory', 'probeSound_idx',
       'Trial', 'Subject', 'stimulusSounds_name', 'probeSound_name'
]]

Getting rid of additional lists containing lists

In [ ]:
block = eeg_.block.values
new_block = []
for i in block:
  while type(i) is np.ndarray:
    i = i[0]
  new_block.append(i)

In [ ]:
stim_cat = eeg_.StimulusCategory.values
new_stim = []
for i in stim_cat:
  if 'environment' in i:
    new_stim.append('environment')
  elif 'nonwords' in i:
    new_stim.append('nonwords')
  elif 'words' in i:
    new_stim.append('words')

In [ ]:
rt = eeg_.ReactionTime.values
new_rt = []
for i in rt:
  while type(i) is np.ndarray:
    i = i[0]
  new_rt.append(i)

In [ ]:
acc = eeg_.RespCorrect.values
new_acc = []
for i in acc:
  while type(i) is np.ndarray:
    i = i[0]
  new_acc.append(i)

In [ ]:
resp = eeg_.Resp.values
new_resp = []
for i in resp:
  while type(i) is np.ndarray:
    i = i[0]
  new_resp.append(i)

In [ ]:
probe = eeg_.ProbeCategory.values
new_probe = []
for i in probe:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe.append(i)

In [ ]:
probe_idx = eeg_.probeSound_idx.values
new_probe_idx = []
for i in probe_idx:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_idx.append(i)

In [ ]:
stim_idx = eeg_.stimulusSounds_idx.values
new_stim_idx = []
for i in stim_idx:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0])
  new_stim_idx.append(element)

In [ ]:
eeg_['block'] = new_block
eeg_['StimulusCategory'] = new_stim
eeg_['ReactionTime'] = new_rt
eeg_['RespCorrect'] = new_acc
eeg_['Resp'] = new_resp
eeg_['ProbeCategory'] = new_probe
eeg_['probeSound_idx'] = new_probe_idx
eeg_['stimulusSounds_idx'] = new_stim_idx

<ipython-input-58-ea534a877bb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['block'] = new_block
<ipython-input-58-ea534a877bb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['StimulusCategory'] = new_stim
<ipython-input-58-ea534a877bb7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  if len(i) not in length:
    length.append(len(i))
print(length)

[3, 7, 9, 5]


In [ ]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  length.append(len(i))
eeg_['Length'] = length

<ipython-input-60-d50296b58efb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Length'] = length


In [ ]:
words = []
nonwords = []
environment = []
for i in eeg_.StimulusCategory.values:
  if i == 'nonwords':
    words.append(0)
    nonwords.append(1)
    environment.append(0)
  elif i == 'words':
    words.append(1)
    nonwords.append(0)
    environment.append(0)
  elif i == 'environment':
    words.append(0)
    nonwords.append(0)
    environment.append(1)

In [ ]:
eeg_['Words'] = words
eeg_['Nonwords'] = nonwords
eeg_['Environment'] = environment

<ipython-input-62-3c31c9b484ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Words'] = words
<ipython-input-62-3c31c9b484ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Nonwords'] = nonwords
<ipython-input-62-3c31c9b484ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [ ]:
probe_names = eeg_.probeSound_name.values
new_probe_names = []
for name in probe_names:
  name = name[0][0][0]
  new_probe_names.append(name)

In [ ]:
stimulus_names = eeg_.stimulusSounds_name.values
new_stim_names = []
for name in stimulus_names:
  name = name[0][0]
  new_name = []
  for j in name:
    new_name.append(j[0][0])
  new_stim_names.append(new_name)

In [ ]:
new_new_stim_names = []
for element in new_stim_names:
  new_element = []
  for name in element:
    new_element.append(name[:-4])
  new_new_stim_names.append(new_element)

In [ ]:
new_new_probe_names = []
for i in new_probe_names:
  new_new_probe_names.append(i[:-4])

In [ ]:
eeg_['stimulusSounds_name'] = new_new_stim_names
eeg_['probeSound_name'] = new_new_probe_names

<ipython-input-67-a747f04adf7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['stimulusSounds_name'] = new_new_stim_names
<ipython-input-67-a747f04adf7e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['probeSound_name'] = new_new_probe_names


**Adding frequency and imaginabilty scores**

In [ ]:
category = eeg_.StimulusCategory.values.tolist()
probe = eeg_.probeSound_name.values
stimuli = eeg_.stimulusSounds_name.values
standard = all_stim[all_stim['Category'] == 'word']['Stimulus'].values.tolist()
standard_freq = all_stim[all_stim['Category'] == 'word']['Frequency'].values.tolist()
standard_imag = all_stim[all_stim['Category'] == 'word']['Imaginability'].values.tolist()

stim_freq = []
stim_imag = []
probe_freq = []
probe_imag = []

for i in range(0, eeg_.shape[0]):
  if category[i] == 'words':
    for name in standard:
      if probe[i] == name:
        freq = standard_freq[standard.index(name)]
        imag = standard_imag[standard.index(name)]
        probe_freq.append(freq)
        probe_imag.append(imag)

    values_freq = []
    values_imag = []      
    for lis in stimuli[i]:
      for name in standard:
        if lis == name:
          freq = standard_freq[standard.index(name)]
          imag = standard_imag[standard.index(name)]
          values_freq.append(freq)
          values_imag.append(imag)
    stim_freq.append(values_freq)
    stim_imag.append(values_imag)
  
  else:
    stim_freq.append(np.nan)
    stim_imag.append(np.nan)
    probe_freq.append(np.nan)
    probe_imag.append(np.nan)

In [ ]:
print(eeg_.shape)
print(len(stim_freq))
print(len(stim_imag))
print(len(probe_freq))
print(len(probe_imag))

(2992, 16)
2992
2992
2992
2992


In [ ]:
eeg_['ProbeFreq'] = probe_freq
eeg_['ProbeImag'] = probe_imag
eeg_['StimFreq'] = stim_freq
eeg_['StimImag'] = stim_imag

<ipython-input-70-c1691c97d5de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['ProbeFreq'] = probe_freq
<ipython-input-70-c1691c97d5de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['ProbeImag'] = probe_imag
<ipython-input-70-c1691c97d5de>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
# saving the file
eeg_.to_csv(
    Path(working_path).joinpath('eeg_envir_processed.csv')
)